In [51]:
# define rooms and items
import random
import time


sunny = {
    "name": "sunny",
    "type": "obstacle",
}

rainy = {
    "name": "rainy",
    "type": "obstacle",
}

snowing = {
    "name": "snowing",
    "type": "obstacle",
}

sunglasses = {
    "name": "sunglasses",
    "type": "object",
    "target": sunny,
}

umbrella = {
    "name": "umbrella",
    "type": "object",
    "target": rainy,
}

snow_boots = {
    "name": "snow boots",
    "type": "object",
    "target": snowing,
}

obstacle = {
    "sunny": "sunglasses",
    "rainy": "umbrella",
    "snowing": "snow_boots",
}

coat_hanger = {
    "name": "coat hanger",
    "type": "furniture",
}

couch = {
    "name": "couch",
    "type": "furniture",
}

queen_bed = {
    "name": "queen bed",
    "type": "furniture",
}

double_bed = {
    "name": "double bed",
    "type": "furniture",
}

dresser = {
    "name": "dresser",
    "type": "furniture",
}

dining_table = {
    "name": "dining table",
    "type": "furniture",
}

doors_of_durin = {
    "name": "doors of durin",
    "type": "door",
}

door_b = {
    "name": "door b",
    "type": "door",
}

door_c = {
    "name": "door c",
    "type": "door",
}

bridge = {
    "name": "bridge",
    "type": "door",
}

key_a = {
    "name": "key for door a",
    "type": "key",
    "target": door_a,
}

key_b = {
    "name": "key for door b",
    "type": "key",
    "target": door_b,
}

key_c = {
    "name": "key for door c",
    "type": "key",
    "target": door_c,
}

key_d = {
    "name": "pass for the bridge",
    "type": "key",
    "target": bridge,
}

piano = {
    "name": "piano",
    "type": "furniture",
}

moria = {
    "name": "moria",
    "type": "room",
}

bedroom_1 = {
    "name": "bedroom 1",
    "type": "room",
}

bedroom_2 = {
    "name": "bedroom 2",
    "type": "room",
}

the_bridge_of_khazad_dûm = {
    "name": "the bridge of khazad-dûm",
    "type": "room",
}

outside = {
  "name": "outside"
}


all_rooms = [moria, bedroom_1, bedroom_2, the_bridge_of_khazad_dûm, outside]

all_doors = [doors_of_durin, door_b, door_c, bridge]

# define which items/rooms are related

object_relations = {
    "outside": [bridge],
    "moria": [doors_of_durin],
    "queen bed": [key_b],
    "door c": [bedroom_1, the_bridge_of_khazad_dûm],
    "bridge": [outside, the_bridge_of_khazad_dûm],
    "bedroom 1": [door_a, door_b, door_c, queen_bed],
    "bedroom 2": [double_bed, dresser, door_b],
    "double bed": [key_c],
    "dresser": [key_d],
    "doors of durin": [moria, bedroom_1],
    "door b": [bedroom_1, bedroom_2],
    "the bridge of khazad-dûm": [dining_table, coat_hanger, door_c, bridge],
    "dining table": [sunglasses],
    "couch": [snow_boots],
    "coat hanger": [umbrella]
}

# define game state. Do not directly change this dict. 
# Instead, when a new game starts, make a copy of this
# dict and use the copy to store gameplay state. This 
# way you can replay the game multiple times.

INIT_GAME_STATE = {
    "current_room": moria,
    "keys_collected": [],
    "items_collected": [],
    "target_room": outside,
    "level": [],
    "able_to_collect": 0
}


In [ ]:
def printmap(room):
    """
    Print a tiny map of the house
    with the current position marked
    """
    print("+------+-------------+----D---+")
    print("|      |             |        |")
    print("|      |  BEDROOM 2  |        |")
    if room == bedroom_2:
        print("| MORIA |      *      | KHAZAD |")
    else:
        print("|MORIA |             | KHAZAD |")
    print("|ENTRA |------B------+  DOOM  |")
    if room == moria:
        print("|  *   |             |        |")
    elif room == the_bridge_of_khazad_dûm:
        print("|      |             |    *   |")
    else: print("|      |             |        |")
    print("|      A  BEDROOM 1  C        |")
    if room == bedroom_1:
        print("|      |      *      |        |")
    else: print("|      |             |        |")
    print("+------+-------------+--------+")


    
start_time = time.time()
total_time = 300

def timeout():
    t = time.time()
    elapsed = t-start_time
    minutes = int((total_time-elapsed)/60)
    seconds = int(total_time-elapsed)%60
    
    if(elapsed<total_time) and (minutes+seconds>0):
        minutes = int((total_time-elapsed)/60)
        seconds = int(total_time-elapsed)%60
        print("You have",minutes,"minutes",seconds,"seconds left!!")
        return 1
    else:
        print("You ran out of time! :(\n\nGAME OVER")
        return 0
    

def linebreak():
    """
    Print a line break
    """
    print("\n\n")

def start_game():
    """
    Start the game
    """
    start_time = time.time()
    
    print(" _____                          ______                      \n|  ___|                         | ___ \                     \n| |__ ___  ___ __ _ _ __   ___  | |_/ /___   ___  _ __ ___  \n|  __/ __|/ __/ _` | '_ \ / _ \ |    // _ \ / _ \| '_ ` _ \ \n| |__\__ \ (_| (_| | |_) |  __/ | |\ \ (_) | (_) | | | | | |\n\____/___/\___\__,_| .__/ \___| \_| \_\___/ \___/|_| |_| |_|\n                   | |\n                   |_|\n\n")
    print("You embark on the mystical journey through the Middle Earth. You shall not pass!")
    print("You have five minutes!!")
    play_room(game_state["current_room"])

def play_room(room):
    """
    Play a room. First check if the room being played is the target room.
    If it is, the game will end with success. If the player is in the game 
    room, he should say 'Mellon', which means friend in the elvish language.  
    Otherwise, let player either explore (list all items in this room) 
    or examine an item found here.
    """
    game_state["current_room"] = room
    if(game_state["current_room"] == game_state["target_room"]):
        print("Congrats! You escaped the room!")
    
    elif (room == moria):
        linebreak()
        print("You are standing before The Doors of Durin, entrance to the great Dwarven city of Khazad-dûm. As the moon descends, the engraved text appears. It is Elvish.")
        timecheck = timeout()
        if timecheck==0:
            return
        
        intended_action = input("Speak, friend, and enter. I, Narvi, made them. Celebrimbor of Hollin drew these signs.").strip().lower()
        if intended_action == "mellon":
            game_state["current_room"] = the_bridge_of_khazad_dûm
            game_state["keys_collected"] = [key_a, key_b, key_c, key_d]
            game_state["items_collected"] = []
            play_room(game_state["current_room"])
        
        else:
            print("Here's a clue, refer to Lord of the Rings.")
            play_room(room)
        linebreak()
    
    #elif (room == the_bridge_of_khazad_dûm):
        #print("You are standing before Balrog, a create of the ancient world, long forgotten. ")
        
    
    else:
        
        
        linebreak()
        print("You are now in " + room["name"].upper() + ".")
        timecheck = timeout()
        if timecheck==0:
            return
        
        intended_action = input("What would you like to do? Type 'explore', 'examine' or 'my items'? ").strip().lower()
        if intended_action == "explore":
            explore_room(room)
            play_room(room)
        elif intended_action == "examine":
            examine_item(input("What would you like to examine? ").strip().lower())
        elif intended_action == "my items":
            print("You have:")
            if len(game_state["keys_collected"])==0:
                print("Empty pockets... :(")
            else:
                print(", ".join([item["name"].upper() for item in game_state["keys_collected"]]))
                print(", ".join([item["name"].upper() for item in game_state["items_collected"]]))
            play_room(game_state["current_room"])
            
        # CHEAT TO REACH LAST ROOM
        elif intended_action == "skip":
            game_state["current_room"] = the_bridge_of_khazad_dûm
            game_state["keys_collected"] = [key_a, key_b, key_c, key_d]
            game_state["items_collected"] = [sunglasses, snow_boots, umbrella]
            play_room(game_state["current_room"])
            
        else:
            print("Not sure what you mean. Type 'explore' or 'examine'. ")
            play_room(room)
        linebreak()

def explore_room(room):
    """
    Explore a room. List all items belonging to this room.
    """
    items = [i["name"].upper() for i in object_relations[room["name"]]]
    linebreak()
    printmap(room)
    print("You explore the room. This is " + room["name"] + ". You find " + ", ".join(items))

def get_next_room_of_door(door, current_room):
    """
    From object_relations, find the two rooms connected to the given door.
    Return the room that is not the current_room.
    """
    connected_rooms = object_relations[door["name"]]
    for room in connected_rooms:
        if(not current_room == room):
            return room

def get_item_needed():
    if len(game_state["level"])>0:
        obstacle_out = game_state["level"][0]
        return obstacle_out, obstacle.get(obstacle_out)
    else:
        while True:
            l = input("You are standing before Balrog, a creature of the ancient world, long forgotten. To pass, you need specify your current level on a scale from 0 to 10").strip()
            if l.isdigit():
                l = float(l)
                break
            
        lst = list(obstacle.keys())
        if -1<l<4:
            #print("It's SNOWING!")
            obstacle_out = "snowing"
            game_state["level"].append(obstacle_out)
            return obstacle_out, obstacle.get(obstacle_out)
        elif 8>l>3:
            #print("It's SUNNY!")
            obstacle_out = "sunny"
            game_state["level"].append(obstacle_out)
            return obstacle_out, obstacle.get(obstacle_out)
        elif 7<l<11:
            #print("It's RAINY!")
            obstacle_out = "rainy"
            game_state["level"].append(obstacle_out)
            return obstacle_out, obstacle.get(obstacle_out)
        else:
            print("It's got to be between 0 and 10.")
            get_item_needed()

        
def examine_item(item_name):
    """
    Examine an item which can be a door or furniture.
    First make sure the intended item belongs to the current room.
    Then check if the item is a door. Tell player if key hasn't been 
    collected yet. Otherwise ask player if they want to go to the next
    room. If the item is not a door, then check if it contains keys.
    Collect the key if found and update the game state. At the end,
    play either the current or the next room depending on the game state
    to keep playing.
    """
    current_room = game_state["current_room"]
    next_room = ""
    output = None
    linebreak()
    printmap(current_room)
    
    for item in object_relations[current_room["name"]]:
        if(item["name"] == item_name):
            output = "You examine " + item_name.upper() + ". "
            if(item["type"] == "door"):
                have_key = False
                for key in game_state["keys_collected"]:
                    if(key["target"] == item):
                        have_key = True
                if(have_key):
                    output += "You unlock it with a key you have."
                    next_room = get_next_room_of_door(item, current_room)
                else:
                    output += "It is locked but you don't have the key."
            else:
                if(game_state["able_to_collect"] and item["name"] in object_relations and len(object_relations[item["name"]])>0):
                    item_found = object_relations[item["name"]].pop()
                    if item_found["type"] == "key":
                        game_state["keys_collected"].append(item_found)
                    else:
                        game_state["items_collected"].append(item_found)
                    output += "You find " + item_found["name"].upper() + "."
                else:
                    output += "There isn't anything interesting about it."
            print(output)
            break

    if(output is None):
        print("The item you requested is not found in the current room.")
    
    if(next_room and input("Do you want to go to the next room? Enter 'yes' or 'no' ").strip() == 'yes'):
        if(next_room == game_state["target_room"]):
            game_state["able_to_collect"] = 1
            _, item_needed = get_item_needed()
            have_gear = False
            if item_needed in [item["name"] for item in game_state["items_collected"]]:
                print("Luckily you have your gear and you are ready for the obstacle!")
                play_room(next_room)
            else:
                print("It's",game_state["level"][0].upper() + "!","You can't leave! You should go back to look for proper gear!")
                play_room(current_room)

            
        else:
            play_room(next_room)
    else:
        play_room(current_room)
        
        
        
game_state = INIT_GAME_STATE.copy()

start_game()

 _____                          ______                      
|  ___|                         | ___ \                     
| |__ ___  ___ __ _ _ __   ___  | |_/ /___   ___  _ __ ___  
|  __/ __|/ __/ _` | '_ \ / _ \ |    // _ \ / _ \| '_ ` _ \ 
| |__\__ \ (_| (_| | |_) |  __/ | |\ \ (_) | (_) | | | | | |
\____/___/\___\__,_| .__/ \___| \_| \_\___/ \___/|_| |_| |_|
                   | |
                   |_|


You embark on the mystical journey through the Middle Earth. You shall not pass!
You have five minutes!!



You are standing before The Doors of Durin, entrance to the great Dwarven city of Khazad-dûm. As the moon descends, the engraved text appears. It is Elvish.
You have 4 minutes 59 seconds left!!
Speak, friend, and enter. I, Narvi, made them. Celebrimbor of Hollin drew these signs.mellon



You are now in THE BRIDGE OF KHAZAD-DÛM.
You have 4 minutes 57 seconds left!!
What would you like to do? Type 'explore', 'examine' or 'my items'? examine
What would you like to examine? br